In [47]:

from pathlib import Path
import pandas as pd
import numpy as np

In [48]:
# check the missed countries.


data = pd.read_csv('UNICEF_percentage_and_population_levels_groups20220209.csv')

data = data[['country','Total_Never entered primary','Total_Dropped out of primary','Total_Still attending primary',
             'Total_Did not transition to lower secondary','Total_Dropped out of lower secondary','Total_Still attending lower secondary',
             'Total_Did not transition to upper secondary',          
             'Total_population_agerange']]

data 


,country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984
...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968
93,Viet Nam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025


The UNICEF pathway data doesn't include the 'Dropped out of upper secondary'.
The data is percentage data. We need to multiply the total population in the age range, Unit 1000.

These step-by-step subs make each levels only include the children that **specifically** leave at this level

# Here we get the population data in needs, unit as one children

In [49]:
cats = data.columns.tolist()[1:-1]
print(cats)

for cat in cats:
    data[f'{cat}_pop'] = data[f'{cat}']/100 * data['Total_population_agerange'] * 1000
# percentage for 100, and 1000 as the unit for population 

pop_df = data 

 
 
# take out the dropout one, 
# same for upper secondary 

['Total_Never entered primary', 'Total_Dropped out of primary', 'Total_Still attending primary', 'Total_Did not transition to lower secondary', 'Total_Dropped out of lower secondary', 'Total_Still attending lower secondary', 'Total_Did not transition to upper secondary']


In [50]:
# check the negative case
(pop_df.iloc[:,1:] < 0).any()


Total_Never entered primary                        False
Total_Dropped out of primary                       False
Total_Still attending primary                      False
Total_Did not transition to lower secondary        False
Total_Dropped out of lower secondary               False
Total_Still attending lower secondary              False
Total_Did not transition to upper secondary        False
Total_population_agerange                          False
Total_Never entered primary_pop                    False
Total_Dropped out of primary_pop                   False
Total_Still attending primary_pop                  False
Total_Did not transition to lower secondary_pop    False
Total_Dropped out of lower secondary_pop           False
Total_Still attending lower secondary_pop          False
Total_Did not transition to upper secondary_pop    False
dtype: bool

# here is the duration of education for each level


In [51]:
dur_data = pd.read_csv('years_of_school_levels.csv') 
ind_list = ['Theoretical duration of primary education (years)','Theoretical duration of lower secondary education (years)','Theoretical duration of upper secondary education (years)']
bool_list = dur_data.Indicator.isin(ind_list)
dur_data = dur_data[bool_list]


levels = ['primary','lower secondary','upper secondary']
for level in levels:
    
    dur_group_tmp = dur_data[['Country','Indicator','Time']]
    dur_group_tmp = dur_group_tmp[dur_group_tmp['Indicator']==f'Theoretical duration of {level} education (years)']
    
    
    dur_group = dur_group_tmp.groupby('Country').max(numeric_only=False)
    
    if level == 'primary': 
        dur_df = dur_group.reset_index()
        dur_df = dur_df.merge(dur_data,on=['Country','Time','Indicator'])
        dur_df=dur_df[['Country','Value']]
        dur_df=dur_df.rename({'Value':f'{level}_dur'},axis='columns')
    else:
        dur_df_tmp = dur_group.reset_index()
        dur_df_tmp = dur_df_tmp.merge(dur_data,on=['Country','Time','Indicator'])
        dur_df_tmp = dur_df_tmp[['Country','Value']]
        dur_df_tmp=dur_df_tmp.rename({'Value':f'{level}_dur'},axis='columns')
        dur_df = dur_df.merge(dur_df_tmp,on='Country')
dur_df
# dur_df['primary_need_dur'] = dur_df['primary_dur'] + dur_df['lower secondary_dur'] +dur_df['upper secondary_dur'] 
# dur_df['lower secondary_need_dur'] =  dur_df['lower secondary_dur'] +dur_df['upper secondary_dur'] 
# dur_df['upper secondary_need_dur'] =  dur_df['upper secondary_dur'] 
# dur_df['drop_primary_need_dur'] = dur_df['primary_dur']/2 + dur_df['lower secondary_dur'] +dur_df['upper secondary_dur'] 
# dur_df['drop_lower secondary_need_dur'] = dur_df['lower secondary_dur']/2 +dur_df['upper secondary_dur'] 
# # use the demical half year 
# dur_df

,Country,primary_dur,lower secondary_dur,upper secondary_dur
0,Afghanistan,6.0,3.0,3.0
1,Albania,5.0,4.0,3.0
2,Algeria,5.0,4.0,3.0
3,American Samoa,6.0,4.0,2.0
4,Andorra,6.0,4.0,2.0
...,...,...,...,...
219,Viet Nam,5.0,4.0,3.0
220,Western Sahara,5.0,3.0,3.0
221,Yemen,6.0,3.0,3.0
222,Zambia,7.0,2.0,3.0


# combine the duration data and filled the default


In [52]:
pop_df.rename({
    'country':'Country'
},axis=1,inplace=True)
main_df = pop_df.merge(dur_df,how='left',on='Country')
display(main_df)

main_df['primary_dur'] = main_df['primary_dur'].fillna(6)
main_df['lower secondary_dur'] = main_df['lower secondary_dur'].fillna(3)
main_df['upper secondary_dur'] = main_df['upper secondary_dur'].fillna(3)


,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,Total_Dropped out of primary_pop,Total_Still attending primary_pop,Total_Did not transition to lower secondary_pop,Total_Dropped out of lower secondary_pop,Total_Still attending lower secondary_pop,Total_Did not transition to upper secondary_pop,primary_dur,lower secondary_dur,upper secondary_dur
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,229310.741254,94888.643300,105183.949666,110470.039840,404361.164774,47416.496566,6.0,3.0,3.0
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,1070.629331,148.738857,557.865393,1092.956242,72.909191,14806.229681,5.0,4.0,3.0
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,58303.808936,7120.833048,7971.441581,230048.138658,480804.872336,50443.852822,5.0,4.0,3.0
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,214275.037000,503656.788000,35344.336000,72897.693000,872563.295000,17672.168000,6.0,3.0,3.0
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,14923.888000,31979.760000,29847.776000,130051.024000,624671.312000,34111.744000,6.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,1897.584000,1167.744000,7006.464000,15472.608000,50650.896000,5838.720000,6.0,3.0,3.0
93,Viet Nam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,62866.080000,3929.130000,47149.560000,330046.920000,192527.370000,310401.270000,5.0,4.0,3.0
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,301798.574000,115624.129000,119543.591000,109744.936000,460536.785000,33315.427000,6.0,3.0,3.0
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,205783.971541,166023.485148,99237.374502,58752.962249,302423.654225,69751.324831,7.0,2.0,3.0


The drop out of school children need *half* of current session and all session later. The children-in-need need all the year from now on. 

# Currently, not GINI data, but remember to time 50% discount for the GDP earning.

# load the GDP values, Units are Dollar per Capita

In [53]:
gdp_data = pd.read_excel('/Users/mengliu/Fabdata/UNICEF_final/return_from_pathway/EAR_4MMN_CUR_NB_A_EN.xlsx',skiprows=[0,1,2,3,4]) 
display(gdp_data)

gdp_data = gdp_data[gdp_data['U.S. dollars'].notna()]
display(gdp_data)

gdp_data = gdp_data.groupby('Reference area')['U.S. dollars'].last()
display(gdp_data)

# gpd_data_value = gdp_data.iloc[:,4:].T
# gpd_data_value=gpd_data_value.dropna(axis=1,how='all')
# gdp_data['GDP']=gpd_data_value.apply(lambda x: x[x.notnull()].values[-1]).T
# gdp_df = gdp_data[['Country Name','GDP']]
# gdp_df

,Reference area,Source,Time,Local currency,2017 PPP $,U.S. dollars,Unnamed: 6
0,Afghanistan,ADM - Labour legislation,2010,4000.0,258.23,86.11,Notes\n\nMonthly minimum wage \n-Currency unit...
1,Afghanistan,ADM - Labour legislation,2011,5000.0,297.83,106.96,Notes\n\nMonthly minimum wage \n-Currency unit...
2,Afghanistan,ADM - Labour legislation,2012,5000.0,292.57,98.19,Notes\n\nMonthly minimum wage \n-Currency unit...
3,Afghanistan,ADM - Labour legislation,2013,5000.0,283.19,90.29,Notes\n\nMonthly minimum wage \n-Currency unit...
4,Afghanistan,ADM - Labour legislation,2015,5000.0,290.79,81.77,Notes\n\nMonthly minimum wage \n-Currency unit...
...,...,...,...,...,...,...,...
1036,Zambia,ADM - Labour legislation,2010,268000.0,NaN,NaN,Notes\n\nMonthly minimum wage \n-Currency unit...
1037,Zambia,ADM - Labour legislation,2011,420000.0,NaN,NaN,Notes\n\nMonthly minimum wage \n-Currency unit...
1038,Zambia,ADM - Labour legislation,2012,700.0,NaN,NaN,Notes\n\nMonthly minimum wage \n-Currency unit...
1039,Zambia,ADM - Labour legislation,2013,700.0,NaN,NaN,Notes\n\nMonthly minimum wage \n-Currency unit...


,Reference area,Source,Time,Local currency,2017 PPP $,U.S. dollars,Unnamed: 6
0,Afghanistan,ADM - Labour legislation,2010,4000.0,258.23,86.11,Notes\n\nMonthly minimum wage \n-Currency unit...
1,Afghanistan,ADM - Labour legislation,2011,5000.0,297.83,106.96,Notes\n\nMonthly minimum wage \n-Currency unit...
2,Afghanistan,ADM - Labour legislation,2012,5000.0,292.57,98.19,Notes\n\nMonthly minimum wage \n-Currency unit...
3,Afghanistan,ADM - Labour legislation,2013,5000.0,283.19,90.29,Notes\n\nMonthly minimum wage \n-Currency unit...
4,Afghanistan,ADM - Labour legislation,2015,5000.0,290.79,81.77,Notes\n\nMonthly minimum wage \n-Currency unit...
...,...,...,...,...,...,...,...
1031,Viet Nam,ADM - Labour legislation,2016,3500000.0,453.58,159.56,Notes\n\nMonthly minimum wage \n-Currency unit...
1032,Viet Nam,ADM - Labour legislation,2017,3750000.0,480.30,167.63,Notes\n\nMonthly minimum wage \n-Currency unit...
1033,Viet Nam,ADM - Labour legislation,2018,3980000.0,504.36,176.09,Notes\n\nMonthly minimum wage \n-Currency unit...
1034,Viet Nam,ADM - Labour legislation,2019,4180000.0,524.63,181.34,Notes\n\nMonthly minimum wage \n-Currency unit...


Reference area
Afghanistan         70.75
Albania            239.30
Algeria            226.79
Andorra           1082.74
Angola              58.81
                   ...   
Ukraine            175.20
United Kingdom    1852.13
United States     1256.67
Uzbekistan          21.34
Viet Nam           190.45
Name: U.S. dollars, Length: 132, dtype: float64

# The return rates are the increase of salery **from previous year** for the two levels. 
From the paper

In [54]:
return_data = pd.read_excel('Comparable Returns to Education Database.xlsx',sheet_name= 'Returns Database') 
maxyear = return_data.groupby(['Economy'],sort=False)['Year'].max()
return_data = return_data[['Economy','Year','Pri','Sec',' β1']]
return_data = return_data.merge(maxyear,on=['Economy','Year'])
# if the secondary data is empty, fill with same as the primary
return_data['Sec'].fillna(return_data[' β1'],inplace=True)
return_data = return_data[['Economy','Year','Pri','Sec']]
return_data.rename({'Pri':'primary_return_rate','Sec':'secondary_return_rate'},axis='columns',inplace=True)
return_df = return_data.dropna()
return_df

,Economy,Year,primary_return_rate,secondary_return_rate
0,Afghanistan,2007,4.110700,1.552420
1,Albania,2005,14.555667,2.877140
2,Argentina,2012,3.715300,4.690300
3,Armenia,2008,39.688335,0.829500
4,Australia,2010,13.997901,20.267717
...,...,...,...,...
136,West Bank and Gaza,2008,8.101767,0.002800
138,South Africa,2011,8.879499,12.272751
139,"Congo, Dem. Rep.",2004,8.990500,1.689300
140,Zambia,2012,13.316266,12.065867


# adding the GDP data

In [55]:
gdp_df = gdp_data.to_frame().reset_index()
display(gdp_df)
replace_gdp={'Egypt, Arab Rep.':'Egypt','Congo, Dem. Rep.':'Democratic Republic of the Congo',
             'Congo, Rep.':'Congo',
             'Lao PDR':'Lao People\'s Democratic Republic',
             'St. Lucia':'Saint Lucia',
             'Yemen, Rep.' : 'Yemen'
             }

replace_pop={'C√¥te d\'Ivoire':'Cote d\'Ivoire',
             'Viet Nam':'Vietnam'}
gdp_df = gdp_df.replace(replace_gdp)
display(gdp_df)






main_df = main_df.replace(replace_pop)
main_df = main_df.merge(gdp_df,how='left',left_on='Country',right_on='Reference area')
main_df

,Reference area,U.S. dollars
0,Afghanistan,70.75
1,Albania,239.30
2,Algeria,226.79
3,Andorra,1082.74
4,Angola,58.81
...,...,...
127,Ukraine,175.20
128,United Kingdom,1852.13
129,United States,1256.67
130,Uzbekistan,21.34


,Reference area,U.S. dollars
0,Afghanistan,70.75
1,Albania,239.30
2,Algeria,226.79
3,Andorra,1082.74
4,Angola,58.81
...,...,...
127,Ukraine,175.20
128,United Kingdom,1852.13
129,United States,1256.67
130,Uzbekistan,21.34


,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,...,Total_Still attending primary_pop,Total_Did not transition to lower secondary_pop,Total_Dropped out of lower secondary_pop,Total_Still attending lower secondary_pop,Total_Did not transition to upper secondary_pop,primary_dur,lower secondary_dur,upper secondary_dur,Reference area,U.S. dollars
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,...,94888.643300,105183.949666,110470.039840,404361.164774,47416.496566,6.0,3.0,3.0,Afghanistan,70.75
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,...,148.738857,557.865393,1092.956242,72.909191,14806.229681,5.0,4.0,3.0,Albania,239.30
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,...,7120.833048,7971.441581,230048.138658,480804.872336,50443.852822,5.0,4.0,3.0,Algeria,226.79
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,...,503656.788000,35344.336000,72897.693000,872563.295000,17672.168000,6.0,3.0,3.0,Angola,58.81
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,...,31979.760000,29847.776000,130051.024000,624671.312000,34111.744000,6.0,3.0,3.0,Argentina,239.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,...,1167.744000,7006.464000,15472.608000,50650.896000,5838.720000,6.0,3.0,3.0,NaN,NaN
93,Vietnam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,...,3929.130000,47149.560000,330046.920000,192527.370000,310401.270000,5.0,4.0,3.0,NaN,NaN
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,...,115624.129000,119543.591000,109744.936000,460536.785000,33315.427000,6.0,3.0,3.0,NaN,NaN
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,...,166023.485148,99237.374502,58752.962249,302423.654225,69751.324831,7.0,2.0,3.0,NaN,NaN


# Adding the return data.
default inrease rate is 10%


In [56]:
return_df = return_df.rename({'Economy':'Country'},axis = 1)
data_final=main_df.merge(return_df,on='Country',how='left')
data_final['primary_return_rate'] = data_final['primary_return_rate'].fillna(10.0)
data_final['secondary_return_rate'] = data_final['secondary_return_rate'].fillna(10.0)
data_final

,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,...,Total_Still attending lower secondary_pop,Total_Did not transition to upper secondary_pop,primary_dur,lower secondary_dur,upper secondary_dur,Reference area,U.S. dollars,Year,primary_return_rate,secondary_return_rate
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,...,404361.164774,47416.496566,6.0,3.0,3.0,Afghanistan,70.75,2007.0,4.110700,1.552420
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,...,72.909191,14806.229681,5.0,4.0,3.0,Albania,239.30,2005.0,14.555667,2.877140
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,...,480804.872336,50443.852822,5.0,4.0,3.0,Algeria,226.79,NaN,10.000000,10.000000
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,...,872563.295000,17672.168000,6.0,3.0,3.0,Angola,58.81,NaN,10.000000,10.000000
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,...,624671.312000,34111.744000,6.0,3.0,3.0,Argentina,239.23,2012.0,3.715300,4.690300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,...,50650.896000,5838.720000,6.0,3.0,3.0,NaN,NaN,2012.0,3.993366,4.777500
93,Vietnam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,...,192527.370000,310401.270000,5.0,4.0,3.0,NaN,NaN,2010.0,8.305800,3.264600
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,...,460536.785000,33315.427000,6.0,3.0,3.0,NaN,NaN,NaN,10.000000,10.000000
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,...,302423.654225,69751.324831,7.0,2.0,3.0,NaN,NaN,2012.0,13.316266,12.065867


In [57]:
data_final['increasemental_rate_of_salaries_Never_entered_primary']  = (
                                    ((data_final['primary_return_rate']/100)+1)**data_final['primary_dur'] 
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['lower secondary_dur']
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)

data_final['increasemental_rate_of_Dropped_out_of_primary']  = (
                                    ((data_final['primary_return_rate']/100)+1)**(data_final['primary_dur']/2)
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['lower secondary_dur']
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)

data_final['increasemental_rate_of_Still_in_primary']  = (
                                    ((data_final['primary_return_rate']/100)+1)**(data_final['primary_dur']/2)
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['lower secondary_dur']
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)

data_final['increasemental_rate_of_salaries_Did_not_transition_to_lower_secondary']  =  (
                                    ((data_final['secondary_return_rate']/100)+1)**data_final['lower secondary_dur']
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)

data_final['increasemental_rate_of_salaries_Dropped_out_of_lower_secondary']  =  (
                                    ((data_final['secondary_return_rate']/100)+1)**(data_final['lower secondary_dur']/2)
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)

data_final['increasemental_rate_of_salaries_Still_in_lower_secondary']  =  (
                                    ((data_final['secondary_return_rate']/100)+1)**(data_final['lower secondary_dur']/2)
                                      *((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)


data_final['increasemental_rate_of_Did_not_transition_to_upper_secondary']  = (
                                      ((data_final['secondary_return_rate']/100)+1)**data_final['upper secondary_dur']
                                      * 100)
data_final

,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,...,Year,primary_return_rate,secondary_return_rate,increasemental_rate_of_salaries_Never_entered_primary,increasemental_rate_of_Dropped_out_of_primary,increasemental_rate_of_Still_in_primary,increasemental_rate_of_salaries_Did_not_transition_to_lower_secondary,increasemental_rate_of_salaries_Dropped_out_of_lower_secondary,increasemental_rate_of_salaries_Still_in_lower_secondary,increasemental_rate_of_Did_not_transition_to_upper_secondary
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,...,2007.0,4.110700,1.552420,139.673452,123.773526,123.773526,109.683592,107.178148,107.178148,104.729934
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,...,2005.0,14.555667,2.877140,240.610773,171.306413,171.306413,121.964146,115.237655,115.237655,108.882140
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,...,NaN,10.000000,10.000000,313.842838,247.303640,247.303640,194.871710,161.051000,161.051000,133.100000
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,...,NaN,10.000000,10.000000,313.842838,235.794769,235.794769,177.156100,153.556103,153.556103,133.100000
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,...,2012.0,3.715300,4.690300,163.868557,146.881516,146.881516,131.655397,122.907674,122.907674,114.741186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,...,2012.0,3.993366,4.777500,167.356282,148.807598,148.807598,132.314730,123.369029,123.369029,115.028140
93,Vietnam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,...,2010.0,8.305800,3.264600,186.603099,152.858489,152.858489,125.216128,117.424126,117.424126,110.117008
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,...,NaN,10.000000,10.000000,313.842838,235.794769,235.794769,177.156100,153.556103,153.556103,133.100000
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,...,2012.0,13.316266,12.065867,424.045612,273.772410,273.772410,176.752997,157.722419,157.722419,140.740818


In [58]:

##  around the year before into inter 
##  experience discount  
def retrun_rate_long_term(return_rate,year_before_work,year_of_work=40,discount_rate=10):
    year_before_work = np.ceil(year_before_work)
    return_sum = 0 
    for year in range(int(year_before_work),int(year_before_work)+int(year_of_work)):
        return_this_year = (1+return_rate/100)/(1+(discount_rate/100))**year
        return_sum = return_sum + return_this_year
    return return_sum

In [59]:
data_final['year_before_work_primary_full'] = data_final['primary_dur'] + data_final['lower secondary_dur'] + data_final['upper secondary_dur']
data_final['year_before_work_primary_half'] = data_final['primary_dur']/2 + data_final['lower secondary_dur'] + data_final['upper secondary_dur']
data_final['year_before_work_LS_full'] =  data_final['lower secondary_dur'] + data_final['upper secondary_dur']
data_final['year_before_work_LS_half'] =  data_final['lower secondary_dur']/2 + data_final['upper secondary_dur']
data_final['year_before_work_US_full'] = data_final['upper secondary_dur']


In [60]:
f = retrun_rate_long_term


data_final['increasemental_rate_of_salaries_Never_entered_primary_longterm'] = data_final.apply(lambda x: f(x['increasemental_rate_of_salaries_Never_entered_primary'], x['year_before_work_primary_full']), axis=1) 
data_final['increasemental_rate_of_salaries_Dropped_out_of_primary_longterm']  = data_final.apply(lambda x: f(x['increasemental_rate_of_Dropped_out_of_primary'], x['year_before_work_primary_half']), axis=1)                                                                                                            
data_final['increasemental_rate_of_salaries_Still_attending_primary_longterm'] =  data_final.apply(lambda x: f(x['increasemental_rate_of_Still_in_primary'], x['year_before_work_primary_half']), axis=1)                                                                                            

data_final['increasemental_rate_of_salaries_Did_not_transition_to_lower_secondary_longterm'] = data_final.apply(lambda x: f(x['increasemental_rate_of_salaries_Did_not_transition_to_lower_secondary'], x['year_before_work_LS_full']), axis=1) 
data_final['increasemental_rate_of_salaries_Dropped_out_of_lower_secondary_longterm'] = data_final.apply(lambda x: f(x['increasemental_rate_of_salaries_Dropped_out_of_lower_secondary'], x['year_before_work_LS_half']), axis=1) 
data_final['increasemental_rate_of_salaries_Still_attending_lower_secondary_longterm'] = data_final.apply(lambda x: f(x['increasemental_rate_of_salaries_Still_in_lower_secondary'], x['year_before_work_LS_half']), axis=1) 
data_final['increasemental_rate_of_salaries_Did_not_transition_to_upper_secondary_longterm'] = data_final.apply(lambda x: f(x['increasemental_rate_of_Did_not_transition_to_upper_secondary'], x['year_before_work_US_full']), axis=1)





In [61]:
item_to_cal = ['Total_Never entered primary','Total_Dropped out of primary','Total_Still attending primary',
             'Total_Did not transition to lower secondary','Total_Dropped out of lower secondary','Total_Still attending lower secondary',
             'Total_Did not transition to upper secondary']

for item in item_to_cal:
    item_tmp = item[6:]
    item_tmp = item_tmp.replace(" ","_")
    data_final[f"{item}_return_in_money"] = data_final[f'increasemental_rate_of_salaries_{item_tmp}_longterm'] * data_final['U.S. dollars']* data_final[f'{item}_pop'] * 12
data_final    

,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,...,increasemental_rate_of_salaries_Dropped_out_of_lower_secondary_longterm,increasemental_rate_of_salaries_Still_attending_lower_secondary_longterm,increasemental_rate_of_salaries_Did_not_transition_to_upper_secondary_longterm,Total_Never entered primary_return_in_money,Total_Dropped out of primary_return_in_money,Total_Still attending primary_return_in_money,Total_Did not transition to lower secondary_return_in_money,Total_Dropped out of lower secondary_return_in_money,Total_Still attending lower secondary_return_in_money,Total_Did not transition to upper secondary_return_in_money
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,...,13.837891,13.837891,16.545987,6.560524e+09,1.987449e+09,8.224052e+08,1.136985e+09,1.297843e+09,4.750584e+09,6.660853e+08
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,...,14.376203,14.376203,16.881562,5.526653e+07,3.459280e+07,4.805858e+06,1.962803e+07,4.512019e+07,3.009889e+06,7.177630e+08
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,...,17.436179,17.436179,18.838816,1.493165e+09,2.285462e+09,2.791308e+08,3.531146e+08,1.091629e+10,2.281525e+10,2.586229e+09
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,...,16.935578,16.935578,18.838816,1.702690e+09,2.316504e+09,5.444980e+09,4.197686e+08,8.712569e+08,1.042868e+10,2.349502e+08
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,...,14.888501,14.888501,17.355082,1.107071e+08,4.825282e+08,1.033989e+09,1.205271e+09,5.558552e+09,2.669927e+10,1.699525e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,...,14.919316,14.919316,17.378273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,Vietnam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,...,14.522243,14.522243,16.981363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,...,16.935578,16.935578,18.838816,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,...,18.935241,18.935241,19.456336,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:

data_final['all_return'] = data_final.iloc[:,-7:].sum(axis=1)

In [63]:
display(data_final)
data_final.to_csv('UNICEF_return_model_minimal_wage.csv')

,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,...,increasemental_rate_of_salaries_Still_attending_lower_secondary_longterm,increasemental_rate_of_salaries_Did_not_transition_to_upper_secondary_longterm,Total_Never entered primary_return_in_money,Total_Dropped out of primary_return_in_money,Total_Still attending primary_return_in_money,Total_Did not transition to lower secondary_return_in_money,Total_Dropped out of lower secondary_return_in_money,Total_Still attending lower secondary_return_in_money,Total_Did not transition to upper secondary_return_in_money,all_return
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,...,13.837891,16.545987,6.560524e+09,1.987449e+09,8.224052e+08,1.136985e+09,1.297843e+09,4.750584e+09,6.660853e+08,1.722188e+10
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,...,14.376203,16.881562,5.526653e+07,3.459280e+07,4.805858e+06,1.962803e+07,4.512019e+07,3.009889e+06,7.177630e+08,8.801863e+08
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,...,17.436179,18.838816,1.493165e+09,2.285462e+09,2.791308e+08,3.531146e+08,1.091629e+10,2.281525e+10,2.586229e+09,4.072865e+10
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,...,16.935578,18.838816,1.702690e+09,2.316504e+09,5.444980e+09,4.197686e+08,8.712569e+08,1.042868e+10,2.349502e+08,2.141883e+10
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,...,14.888501,17.355082,1.107071e+08,4.825282e+08,1.033989e+09,1.205271e+09,5.558552e+09,2.669927e+10,1.699525e+09,3.678984e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,...,14.919316,17.378273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
93,Vietnam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,...,14.522243,16.981363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,...,16.935578,18.838816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,...,18.935241,19.456336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00


In [64]:

datapath = Path('/Users/mengliu/Fabdata/Return_models') / 'data' 
total_GDP = pd.read_csv(datapath / 'API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3607744.csv',skiprows=[0,1,2,3])
gpd_data_value = total_GDP.iloc[:,4:].T
gpd_data_value=gpd_data_value.dropna(axis=1,how='all')
total_gdp_data=gpd_data_value.apply(lambda x: x[x.notnull()].values[-1]).T
total_gdp_data
total_GDP['GDP_total'] = total_gdp_data
total_GDP = total_GDP[['Country Name','GDP_total']]
total_GDP

,Country Name,GDP_total
0,Aruba,3.202189e+09
1,Africa Eastern and Southern,9.188155e+11
2,Afghanistan,2.011614e+10
3,Africa Western and Central,7.845876e+11
4,Angola,5.837598e+10
...,...,...
261,Kosovo,7.716925e+09
262,"Yemen, Rep.",2.160614e+10
263,South Africa,3.354421e+11
264,Zambia,1.811063e+10


In [65]:
total_GDP_df = total_GDP.rename({'Country Name':'Country'},axis=1)

data_final = data_final.merge(total_GDP_df,on='Country',how='left')

In [66]:
data_final['%return per GDP'] = data_final['all_return']/data_final['GDP_total']*100
data_final


,Country,Total_Never entered primary,Total_Dropped out of primary,Total_Still attending primary,Total_Did not transition to lower secondary,Total_Dropped out of lower secondary,Total_Still attending lower secondary,Total_Did not transition to upper secondary,Total_population_agerange,Total_Never entered primary_pop,...,Total_Never entered primary_return_in_money,Total_Dropped out of primary_return_in_money,Total_Still attending primary_return_in_money,Total_Did not transition to lower secondary_return_in_money,Total_Dropped out of lower secondary_return_in_money,Total_Still attending lower secondary_return_in_money,Total_Did not transition to upper secondary_return_in_money,all_return,GDP_total,%return per GDP
0,Afghanistan,33.861341,8.254577,3.415739,3.786342,3.976628,14.555927,1.706868,2777.983,940662.293496,...,6.560524e+09,1.987449e+09,8.224052e+08,1.136985e+09,1.297843e+09,4.750584e+09,6.660853e+08,1.722188e+10,2.011614e+10,85.612241
1,Albania,1.099108,0.713800,0.099166,0.371935,0.728686,0.048609,9.871478,149.990,1648.552290,...,5.526653e+07,3.459280e+07,4.805858e+06,1.962803e+07,4.512019e+07,3.009889e+06,7.177630e+08,8.801863e+08,1.488763e+10,5.912199
2,Algeria,2.157483,3.252027,0.397181,0.444625,12.831457,26.817983,2.813620,1792.845,38680.317844,...,1.493165e+09,2.285462e+09,2.791308e+08,3.531146e+08,1.091629e+10,2.281525e+10,2.586229e+09,4.072865e+10,1.450092e+11,28.086945
3,Angola,7.700000,9.700000,22.800000,1.600000,3.300000,39.500000,0.800000,2209.021,170094.617000,...,1.702690e+09,2.316504e+09,5.444980e+09,4.197686e+08,8.712569e+08,1.042868e+10,2.349502e+08,2.141883e+10,5.837598e+10,36.691174
4,Argentina,0.200000,0.700000,1.500000,1.400000,6.100000,29.300000,1.600000,2131.984,4263.968000,...,1.107071e+08,4.825282e+08,1.033989e+09,1.205271e+09,5.558552e+09,2.669927e+10,1.699525e+09,3.678984e+10,3.892881e+11,9.450545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Uruguay,1.400000,1.300000,0.800000,4.800000,10.600000,34.700000,4.000000,145.968,2043.552000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,5.362883e+10,0.000000
93,Vietnam,1.700000,1.600000,0.100000,1.200000,8.400000,4.900000,7.900000,3929.130,66795.210000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,2.711584e+11,0.000000
94,Yemen,10.800000,15.400000,5.900000,6.100000,5.600000,23.500000,1.700000,1959.731,211650.948000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN
95,Zambia,3.524882,16.126955,13.010990,7.777071,4.604374,23.700449,5.466298,1276.025,44978.377455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,1.811063e+10,0.000000


In [67]:
data_final.to_csv('return_UNICEF_minimalwage.csv')


In [68]:
data_final = pd.read_csv('return_UNICEF.csv')

datapath = Path('/Users/mengliu/Fabdata/Return_models') / 'data' 
income_group = pd.read_excel(datapath / 'CLASS.xlsx')
tab_rename = {
    'Congo, Rep.':'Congo',
    'Congo, Dem. Rep.':'Democratic Republic of the Congo',
    'Egypt, Arab Rep.':'Egypt',
    'Lao PDR':'Lao People\'s Democratic Republic',
    'St. Lucia':'Saint Lucia',
    'São Tomé and Principe':'Sao Tome and Principe',
    'Yemen, Rep.':'Yemen'  
}
income_group.replace(tab_rename,inplace=True)
income_group = income_group[['Economy','Income group','Code']]
data_final = data_final.merge(income_group,left_on='country',right_on='Economy',how='left')


FileNotFoundError: [Errno 2] No such file or directory: 'return_UNICEF.csv'

In [117]:
def GDP_to_benefit_poverty(increased_GDP,income_group):
    K ={
     'Low income':0.73,
     'Lower middle income':0.66,
     'Upper middle income':0.51,
     'High income':0.59   
    }
    I = {     
     'Low income':0.455,
     'Lower middle income':0.169,
     'Upper middle income':0.016,
     'High income':0.006   
    }
    N = {     
     'Low income':0.18,
     'Lower middle income':0.043,
     'Upper middle income':0.004,
     'High income':0.004   
    }
    k = K[income_group]
    i = I[income_group]
    j = increased_GDP
    m = i*j*k
    n = N[income_group]
    o = m/(n*1.9*365*(47+8))    
    return o
#Estimated number lifted from extreme poverty
f = GDP_to_benefit_poverty
data_final['benefit_poverty'] = data_final.apply(lambda x: f(x['all_return'], x['Income group']), axis=1) 
data_final

,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,drop_primary_final_longterm,drop_ls_longterm,all_return,Country Name_y,GDP_total,return per GDP,Economy_y,Income group,Code,benefit_poverty
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,10.208547,12.579901,5.482297e+09,Afghanistan,2.011614e+10,27.253230,Afghanistan,Low income,AFG,2.652254e+05
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,11.251815,11.881160,7.959437e+08,Albania,1.488763e+10,5.346342,Albania,Upper middle income,ALB,4.256997e+04
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,14.403627,14.410065,2.472823e+10,Algeria,1.450092e+11,17.052874,Algeria,Lower middle income,DZA,1.681691e+06
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,15.318955,15.395980,2.896226e+10,Angola,5.837598e+10,49.613314,Angola,Lower middle income,AGO,1.969634e+06
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,11.262733,13.535001,5.978412e+10,Argentina,3.892881e+11,15.357295,Argentina,Upper middle income,ARG,3.197473e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,11.350601,13.563015,1.013445e+10,Uruguay,5.362883e+10,18.897390,Uruguay,High income,URY,2.351442e+05
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,11.589796,13.088430,2.004428e+10,Vietnam,2.711584e+11,7.392090,Vietnam,Lower middle income,VNM,1.363149e+06
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,15.318955,15.395980,8.556325e+09,NaN,NaN,NaN,Yemen,Low income,YEM,4.139424e+05
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,17.051495,17.213856,8.601749e+09,Zambia,1.811063e+10,47.495575,Zambia,Lower middle income,ZMB,5.849783e+05


In [118]:
# tax share 


def GDP_to_benefit_revenue(increased_GDP,income_group):
    P ={
     'Low income':0.1207,
     'Lower middle income':0.1441,
     'Upper middle income':0.1545,
     'High income':0.1852   
    }
    p = P[income_group]
    j = increased_GDP
    q = j * p    
    return q
f = GDP_to_benefit_revenue
#Public revenue generated
data_final['benefit_revenue'] = data_final.apply(lambda x: f(x['all_return'], x['Income group']), axis=1) 
data_final = data_final.iloc[:,1:]
data_final.drop('Economy_y',axis=1)
data_final.to_csv('return_UNICEF_benefit.csv')

data_final

,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,Total_Needs in upper secondary_pop,...,drop_ls_longterm,all_return,Country Name_y,GDP_total,return per GDP,Economy_y,Income group,Code,benefit_poverty,benefit_revenue
0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,451777.661229,...,12.579901,5.482297e+09,Afghanistan,2.011614e+10,27.253230,Afghanistan,Low income,AFG,2.652254e+05,6.617133e+08
1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,14879.138872,...,11.881160,7.959437e+08,Albania,1.488763e+10,5.346342,Albania,Upper middle income,ALB,4.256997e+04,1.229733e+08
2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,531248.725172,...,14.410065,2.472823e+10,Algeria,1.450092e+11,17.052874,Algeria,Lower middle income,DZA,1.681691e+06,3.563338e+09
3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,888026.442000,...,15.395980,2.896226e+10,Angola,5.837598e+10,49.613314,Angola,Lower middle income,AGO,1.969634e+06,4.173461e+09
4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,656651.072000,...,13.535001,5.978412e+10,Argentina,3.892881e+11,15.357295,Argentina,Upper middle income,ARG,3.197473e+06,9.236646e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,56489.616000,...,13.563015,1.013445e+10,Uruguay,5.362883e+10,18.897390,Uruguay,High income,URY,2.351442e+05,1.876900e+09
90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,498999.510000,...,13.088430,2.004428e+10,Vietnam,2.711584e+11,7.392090,Vietnam,Lower middle income,VNM,1.363149e+06,2.888380e+09
91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,495811.943000,...,15.395980,8.556325e+09,NaN,NaN,NaN,Yemen,Low income,YEM,4.139424e+05,1.032748e+09
92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,372174.979028,...,17.213856,8.601749e+09,Zambia,1.811063e+10,47.495575,Zambia,Lower middle income,ZMB,5.849783e+05,1.239512e+09


In [119]:

data_final = pd.read_csv('return_UNICEF_benefit.csv')

datapath = Path('/Users/mengliu/Fabdata/Return_models') / 'data' 
hlo_group = pd.read_csv(datapath / 'hlo_Data.csv',na_values='..')
hlo_group = hlo_group[['Country Code','2020 [YR2020]']]
hlo_group = hlo_group.rename({'Country Code':'Code','2020 [YR2020]':'HLO'},axis=1)
data_final = data_final.merge(hlo_group,on='Code',how='left')
data_final

,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,all_return,Country Name_y,GDP_total,return per GDP,Economy_y,Income group,Code,benefit_poverty,benefit_revenue,HLO
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,5.482297e+09,Afghanistan,2.011614e+10,27.253230,Afghanistan,Low income,AFG,2.652254e+05,6.617133e+08,354.758789
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,7.959437e+08,Albania,1.488763e+10,5.346342,Albania,Upper middle income,ALB,4.256997e+04,1.229733e+08,434.127594
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,2.472823e+10,Algeria,1.450092e+11,17.052874,Algeria,Lower middle income,DZA,1.681691e+06,3.563338e+09,374.089081
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,2.896226e+10,Angola,5.837598e+10,49.613314,Angola,Lower middle income,AGO,1.969634e+06,4.173461e+09,325.965485
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,5.978412e+10,Argentina,3.892881e+11,15.357295,Argentina,Upper middle income,ARG,3.197473e+06,9.236646e+09,408.172638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,1.013445e+10,Uruguay,5.362883e+10,18.897390,Uruguay,High income,URY,2.351442e+05,1.876900e+09,437.696625
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,2.004428e+10,Vietnam,2.711584e+11,7.392090,Vietnam,Lower middle income,VNM,1.363149e+06,2.888380e+09,519.100281
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,8.556325e+09,NaN,NaN,NaN,Yemen,Low income,YEM,4.139424e+05,1.032748e+09,321.327026
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,8.601749e+09,Zambia,1.811063e+10,47.495575,Zambia,Lower middle income,ZMB,5.849783e+05,1.239512e+09,358.140442


In [120]:
data_final_tmp = data_final[['Income group','HLO']]
HLO_average_group = data_final_tmp.groupby(['Income group']).mean().reset_index()
for gi in HLO_average_group['Income group'].values:

    data_final.loc[data_final['Income group']==gi,'HLO_average'] = HLO_average_group[HLO_average_group['Income group']==gi]['HLO'].values[0]
data_final

,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,Country Name_y,GDP_total,return per GDP,Economy_y,Income group,Code,benefit_poverty,benefit_revenue,HLO,HLO_average
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,Afghanistan,2.011614e+10,27.253230,Afghanistan,Low income,AFG,2.652254e+05,6.617133e+08,354.758789,355.407968
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,Albania,1.488763e+10,5.346342,Albania,Upper middle income,ALB,4.256997e+04,1.229733e+08,434.127594,413.097653
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,Algeria,1.450092e+11,17.052874,Algeria,Lower middle income,DZA,1.681691e+06,3.563338e+09,374.089081,387.844086
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,Angola,5.837598e+10,49.613314,Angola,Lower middle income,AGO,1.969634e+06,4.173461e+09,325.965485,387.844086
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,Argentina,3.892881e+11,15.357295,Argentina,Upper middle income,ARG,3.197473e+06,9.236646e+09,408.172638,413.097653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,Uruguay,5.362883e+10,18.897390,Uruguay,High income,URY,2.351442e+05,1.876900e+09,437.696625,437.696625
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,Vietnam,2.711584e+11,7.392090,Vietnam,Lower middle income,VNM,1.363149e+06,2.888380e+09,519.100281,387.844086
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,NaN,NaN,NaN,Yemen,Low income,YEM,4.139424e+05,1.032748e+09,321.327026,355.407968
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,Zambia,1.811063e+10,47.495575,Zambia,Lower middle income,ZMB,5.849783e+05,1.239512e+09,358.140442,387.844086


In [121]:
datapath = Path('/Users/mengliu/Fabdata/Return_models') / 'data' 
total_pop = pd.read_csv(datapath / 'Total_population_worldbank.csv',na_values='..')
total_pop = total_pop[['Country Code','2020']]
total_pop = total_pop.rename({'Country Code':'Code','2020':'Total_pop'},axis=1)
data_final_tmp = data_final.merge(total_pop,on='Code',how='left')
data_final_tmp = data_final_tmp[['Income group','Total_pop','HLO']]
data_final['total_pop'] = data_final_tmp.Total_pop
data_final_tmp = data_final_tmp.dropna()
data_final_tmp = data_final_tmp.groupby(['Income group']).apply(lambda x:np.average(x['HLO'],weights=x['Total_pop'])).to_frame(name='weighted_avergage_hlo').reset_index()
data_final_tmp
for gi in data_final_tmp['Income group'].values:
       data_final.loc[data_final['Income group']==gi,'HLO_weighted_average'] = data_final_tmp[data_final_tmp['Income group']==gi].values[0][1]

data_final.to_csv('return_UNICEF_benefit_with_weightHLO.csv')
data_final



,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,return per GDP,Economy_y,Income group,Code,benefit_poverty,benefit_revenue,HLO,HLO_average,total_pop,HLO_weighted_average
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,27.253230,Afghanistan,Low income,AFG,2.652254e+05,6.617133e+08,354.758789,355.407968,38928341.0,350.427762
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,5.346342,Albania,Upper middle income,ALB,4.256997e+04,1.229733e+08,434.127594,413.097653,2837743.0,414.282878
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,17.052874,Algeria,Lower middle income,DZA,1.681691e+06,3.563338e+09,374.089081,387.844086,43851043.0,387.332459
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,49.613314,Angola,Lower middle income,AGO,1.969634e+06,4.173461e+09,325.965485,387.844086,32866268.0,387.332459
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,15.357295,Argentina,Upper middle income,ARG,3.197473e+06,9.236646e+09,408.172638,413.097653,45376763.0,414.282878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,18.897390,Uruguay,High income,URY,2.351442e+05,1.876900e+09,437.696625,437.696625,3473727.0,437.696625
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,7.392090,Vietnam,Lower middle income,VNM,1.363149e+06,2.888380e+09,519.100281,387.844086,97338583.0,387.332459
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,NaN,Yemen,Low income,YEM,4.139424e+05,1.032748e+09,321.327026,355.407968,29825968.0,350.427762
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,47.495575,Zambia,Lower middle income,ZMB,5.849783e+05,1.239512e+09,358.140442,387.844086,18383956.0,387.332459


In [122]:
# LAYS  = number of children * HLO / 650
# 
# devide 650 
data_final['LAYS'] = (data_final['primary_need_dur']*data_final['HLO_weighted_average']/650*data_final['Total_Needs in primary_pop'] +
                      data_final['lower secondary_need_dur']*data_final['HLO_weighted_average']/650*data_final['Total_Needs in lower secondary_pop'] +
                      data_final['upper secondary_need_dur']*data_final['HLO_weighted_average']/650*data_final['Total_Needs in upper secondary_pop'] +
                      data_final['drop_primary_need_dur']*data_final['HLO_weighted_average']/650*data_final['Total_Dropped out of primary_pop'] +
                      data_final['drop_lower secondary_need_dur']*data_final['HLO_weighted_average']/650*data_final['Total_Dropped out of lower secondary_pop']

                      )
data_final



,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,Economy_y,Income group,Code,benefit_poverty,benefit_revenue,HLO,HLO_average,total_pop,HLO_weighted_average,LAYS
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,Afghanistan,Low income,AFG,2.652254e+05,6.617133e+08,354.758789,355.407968,38928341.0,350.427762,8.844057e+06
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,Albania,Upper middle income,ALB,4.256997e+04,1.229733e+08,434.127594,413.097653,2837743.0,414.282878,5.417675e+04
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,Algeria,Lower middle income,DZA,1.681691e+06,3.563338e+09,374.089081,387.844086,43851043.0,387.332459,2.304737e+06
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,Angola,Lower middle income,AGO,1.969634e+06,4.173461e+09,325.965485,387.844086,32866268.0,387.332459,6.075599e+06
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,Argentina,Upper middle income,ARG,3.197473e+06,9.236646e+09,408.172638,413.097653,45376763.0,414.282878,1.983224e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,Uruguay,High income,URY,2.351442e+05,1.876900e+09,437.696625,437.696625,3473727.0,437.696625,2.220413e+05
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,Vietnam,Lower middle income,VNM,1.363149e+06,2.888380e+09,519.100281,387.844086,97338583.0,387.332459,2.774504e+06
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,Yemen,Low income,YEM,4.139424e+05,1.032748e+09,321.327026,355.407968,29825968.0,350.427762,4.656125e+06
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,Zambia,Lower middle income,ZMB,5.849783e+05,1.239512e+09,358.140442,387.844086,18383956.0,387.332459,2.959664e+06


In [123]:
def GDP_to_benefit_lives(LAYS,total_pop):
    b = LAYS
    r = total_pop
    s = 0.0155 * b / r
    t = s * r
    return t
f = GDP_to_benefit_lives
#Public revenue generated
data_final['benefit_lives'] = data_final.apply(lambda x: f(x['LAYS'], x['total_pop']), axis=1) 

data_final

,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,Income group,Code,benefit_poverty,benefit_revenue,HLO,HLO_average,total_pop,HLO_weighted_average,LAYS,benefit_lives
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,Low income,AFG,2.652254e+05,6.617133e+08,354.758789,355.407968,38928341.0,350.427762,8.844057e+06,137082.883596
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,Upper middle income,ALB,4.256997e+04,1.229733e+08,434.127594,413.097653,2837743.0,414.282878,5.417675e+04,839.739617
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,Lower middle income,DZA,1.681691e+06,3.563338e+09,374.089081,387.844086,43851043.0,387.332459,2.304737e+06,35723.424315
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,Lower middle income,AGO,1.969634e+06,4.173461e+09,325.965485,387.844086,32866268.0,387.332459,6.075599e+06,94171.791812
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,Upper middle income,ARG,3.197473e+06,9.236646e+09,408.172638,413.097653,45376763.0,414.282878,1.983224e+06,30739.964856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,High income,URY,2.351442e+05,1.876900e+09,437.696625,437.696625,3473727.0,437.696625,2.220413e+05,3441.639897
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,Lower middle income,VNM,1.363149e+06,2.888380e+09,519.100281,387.844086,97338583.0,387.332459,2.774504e+06,43004.804984
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,Low income,YEM,4.139424e+05,1.032748e+09,321.327026,355.407968,29825968.0,350.427762,4.656125e+06,72169.942653
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,Lower middle income,ZMB,5.849783e+05,1.239512e+09,358.140442,387.844086,18383956.0,387.332459,2.959664e+06,45874.794288


In [124]:
def GDP_to_benefit_early_marriage(LAYS,income_group):
    X={
     'Low income':0.4946,
     'Lower middle income':0.7956,
     'Upper middle income':0.9372,
     'High income':0.9547   
    }
    Y={
     'Low income':0.7618,
     'Lower middle income':0.8923,
     'Upper middle income':0.9764,
     'High income':0.9738   
    }
    x = X[income_group]
    y = Y[income_group]
    
    b = LAYS
    w = b/2
    z = 0.075 * w * x * y 
    return z
f = GDP_to_benefit_early_marriage
#Public revenue generated
data_final['benefit_early_marriage'] = data_final.apply(lambda x: f(x['LAYS'],x['Income group']), axis=1) 

data_final.to_csv('return_UNICEF_benefit_with_all.csv')
data_final

,Unnamed: 0,country,Total_Needs in primary,Total_Needs in lower secondary,Total_Needs in upper secondary,Total_Dropped out of primary,Total_Dropped out of lower secondary,Total_population_agerange,Total_Needs in primary_pop,Total_Needs in lower secondary_pop,...,Code,benefit_poverty,benefit_revenue,HLO,HLO_average,total_pop,HLO_weighted_average,LAYS,benefit_lives,benefit_early_marriage
0,0,Afghanistan,33.861341,7.202081,16.262794,8.254577,3.976628,2777.983,940662.293496,200072.593132,...,AFG,2.652254e+05,6.617133e+08,354.758789,355.407968,38928341.0,350.427762,8.844057e+06,137082.883596,124961.975230
1,1,Albania,1.099108,0.471101,9.920087,0.713800,0.728686,149.990,1648.552290,706.604250,...,ALB,4.256997e+04,1.229733e+08,434.127594,413.097653,2837743.0,414.282878,5.417675e+04,839.739617,1859.106473
2,2,Algeria,2.157483,0.841806,29.631604,3.252027,12.831457,1792.845,38680.317852,15092.274621,...,DZA,1.681691e+06,3.563338e+09,374.089081,387.844086,43851043.0,387.332459,2.304737e+06,35723.424315,61356.180877
3,3,Angola,7.700000,24.400000,40.200000,9.700000,3.300000,2209.021,170094.617000,539001.124000,...,AGO,1.969634e+06,4.173461e+09,325.965485,387.844086,32866268.0,387.332459,6.075599e+06,94171.791812,161743.214787
4,4,Argentina,0.200000,2.900000,30.800000,0.700000,6.100000,2131.984,4263.968000,61827.536000,...,ARG,3.197473e+06,9.236646e+09,408.172638,413.097653,45376763.0,414.282878,1.983224e+06,30739.964856,68055.462047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,Uruguay,1.400000,5.600000,38.700000,1.300000,10.600000,145.968,2043.552000,8174.208000,...,URY,2.351442e+05,1.876900e+09,437.696625,437.696625,3473727.0,437.696625,2.220413e+05,3441.639897,7741.082393
90,90,Vietnam,1.700000,1.300000,12.700000,1.600000,8.400000,3929.130,66795.210000,51078.690000,...,VNM,1.363149e+06,2.888380e+09,519.100281,387.844086,97338583.0,387.332459,2.774504e+06,43004.804984,73862.196689
91,91,Yemen,10.800000,11.900000,25.300000,15.400000,5.600000,1959.731,211650.948000,233207.989000,...,YEM,4.139424e+05,1.032748e+09,321.327026,355.407968,29825968.0,350.427762,4.656125e+06,72169.942653,65788.655371
92,92,Zambia,3.524882,20.788061,29.166747,16.126955,4.604374,1276.025,44978.377459,265260.859599,...,ZMB,5.849783e+05,1.239512e+09,358.140442,387.844086,18383956.0,387.332459,2.959664e+06,45874.794288,78791.499694
